# Loading the libraries

In [258]:
# Dataframe and Plotting libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Machine Learning libraries
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.metrics import log_loss

from sklearn.feature_selection import mutual_info_regression

from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV

pd.set_option('display.max_columns', None)
# from google.colab import files
import warnings
warnings.filterwarnings('ignore')

# Load the datasets

In [188]:
# Testing path
path = '/content/'

In [189]:
# Load the files into a Pandas Dataframe
train = pd.read_csv(path+'Train.csv')
test = pd.read_csv(path+'Test.csv')
ss = pd.read_csv(path+'SampleSubmission.csv')
VariableDefinitions = pd.read_csv(path + 'VariableDefinitions.csv')
extra_data = pd.read_csv(path + 'extra_data.csv')

In [190]:
# Let’s observe the shape of our datasets.
print('Train data shape :', train.shape)
print('Test data shape :', test.shape)

Train data shape : (373, 12)
Test data shape : (558, 11)


In [191]:
len(extra_data.columns) == len(train.columns)

True

In [192]:
#preview the datasets
train.head(2)

,MERCHANT_CATEGORIZED_AT,MERCHANT_NAME,MERCHANT_CATEGORIZED_AS,PURCHASE_VALUE,PURCHASED_AT,IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY,USER_AGE,USER_GENDER,USER_HOUSEHOLD,USER_INCOME,USER_ID,Transaction_ID
0,2022-05-04 10:25:50.588042+00,UONSDA CHURCH MOGERE MWAYO TITHE,Rent / Mortgage,1700,2022-05-04 13:56:00+00,False,NaN,Male,3,10000,ID_ZX4DCF4K,ID_04mk78fa
1,2021-10-25 16:18:38.586837+00,PARK N GO,Transport & Fuel,100,2021-10-24 14:12:00+00,False,25.0,Female,4,90000,ID_U9WZMGJZ,ID_04xkfb07


In [193]:
extra_data.head(2)

,MERCHANT_CATEGORIZED_AT,MERCHANT_NAME,MERCHANT_CATEGORIZED_AS,PURCHASE_VALUE,PURCHASED_AT,IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY,USER_AGE,USER_GENDER,USER_HOUSEHOLD,USER_INCOME,USER_ID,Transaction_ID
0,NaN,MARK NYABAYO,NaN,20,2021-03-22 12:37:00+00,False,NaN,NaN,1,10000,ID_R4T8Q46J,ID_v13grrw3
1,NaN,ACCESS AFYA,NaN,1,2021-07-05 11:07:00+00,False,NaN,NaN,1,10000,ID_R4T8Q46J,ID_rgne8ij5


In [194]:
extra_data["MERCHANT_CATEGORIZED_AS"].value_counts()

Series([], Name: MERCHANT_CATEGORIZED_AS, dtype: int64)

In [195]:
#preview the test dataset
test.head(3)

,MERCHANT_CATEGORIZED_AT,MERCHANT_NAME,PURCHASE_VALUE,PURCHASED_AT,IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY,USER_AGE,USER_GENDER,USER_HOUSEHOLD,USER_INCOME,USER_ID,Transaction_ID
0,2022-06-01 10:25:16.7131+00,KCB PAYBILL AC,150000,2022-05-05 08:29:00+00,True,NaN,Male,5,150000,ID_O8P8YS18,ID_00x9h2yx
1,2022-03-16 13:05:51.851102+00,IPAY LTD,7394,2019-10-05 16:02:00+00,False,NaN,Female,1,10000,ID_40L9OTIM,ID_01db594f
2,2022-03-10 18:25:42.112294+00,JOHN NJIRU GAKUNJU,200,2021-11-25 09:56:00+00,False,NaN,Female,1,85000,ID_1I8XYBWK,ID_07pm99no


The dependent column name is MERCHANT_CATEGORIZED_AS

# EDA 

In [196]:
#Check if there are merchants that accept mpesa send money only
train.groupby(["MERCHANT_NAME", "IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY"])["IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY"].sum().sort_values(ascending=False).head(3)

MERCHANT_NAME           IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY
PETER KENDA             True                                     10
KPLC PREPAID            True                                      8
EQUITY PAYBILL ACCOUNT  True                                      8
Name: IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY, dtype: int64

In [197]:
#Check if majority of purchase value is done by mpesa


In [198]:
#check for missing values
train.isnull().sum()

MERCHANT_CATEGORIZED_AT                    0
MERCHANT_NAME                              0
MERCHANT_CATEGORIZED_AS                    0
PURCHASE_VALUE                             0
PURCHASED_AT                               0
IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY      0
USER_AGE                                 312
USER_GENDER                                6
USER_HOUSEHOLD                             0
USER_INCOME                                0
USER_ID                                    0
Transaction_ID                             0
dtype: int64

In [199]:
print(train.describe())

       PURCHASE_VALUE   USER_AGE  USER_HOUSEHOLD     USER_INCOME
count      373.000000  61.000000      373.000000      373.000000
mean      4506.522788  25.803279        2.831099   146235.924933
std      15722.261797   1.423841        1.600033   166521.780346
min         10.000000  25.000000        1.000000     8000.000000
25%        150.000000  25.000000        1.000000    23000.000000
50%        690.000000  25.000000        3.000000    90000.000000
75%       2000.000000  27.000000        4.000000   200000.000000
max     150000.000000  32.000000        7.000000  1000000.000000


In [200]:
#Fill missing value with mean & mode
train['USER_AGE'].fillna((train['USER_AGE'].mean()), inplace=True)
train['USER_GENDER'].fillna(train['USER_GENDER'].mode()[0], inplace=True)

test['USER_AGE'].fillna((train['USER_AGE'].mean()), inplace=True)
test['USER_GENDER'].fillna(train['USER_GENDER'].mode()[0], inplace=True)

In [201]:
#Confirm no missing values
train.isnull().sum().sum()


0

In [202]:
test.isnull().sum().sum()


0

## Converting Categorical Feature

In [203]:
train.head(3)

,MERCHANT_CATEGORIZED_AT,MERCHANT_NAME,MERCHANT_CATEGORIZED_AS,PURCHASE_VALUE,PURCHASED_AT,IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY,USER_AGE,USER_GENDER,USER_HOUSEHOLD,USER_INCOME,USER_ID,Transaction_ID
0,2022-05-04 10:25:50.588042+00,UONSDA CHURCH MOGERE MWAYO TITHE,Rent / Mortgage,1700,2022-05-04 13:56:00+00,False,25.803279,Male,3,10000,ID_ZX4DCF4K,ID_04mk78fa
1,2021-10-25 16:18:38.586837+00,PARK N GO,Transport & Fuel,100,2021-10-24 14:12:00+00,False,25.000000,Female,4,90000,ID_U9WZMGJZ,ID_04xkfb07
2,2022-05-20 14:17:30.917297+00,SAFARICOM OFFERS TUNUKIWA,Data & WiFi,20,2022-05-27 12:46:00+00,False,25.803279,Male,3,10000,ID_ZX4DCF4K,ID_051urwuw


In [204]:
# Use a dictionary comprehension and zip to create a dictionary for all the categories in the train data
labels_train = train['MERCHANT_CATEGORIZED_AS'].astype('category').cat.categories.tolist()
replace_map_train = {'MERCHANT_CATEGORIZED_AS' : {k: v for k,v in zip(labels_train,list(range(1,len(labels_train)+1)))}}
print("Train data: ", replace_map_train)

Train data:  {'MERCHANT_CATEGORIZED_AS': {'Bills & Fees': 1, 'Data & WiFi': 2, 'Education': 3, 'Emergency fund': 4, 'Family & Friends': 5, 'Going out': 6, 'Groceries': 7, 'Health': 8, 'Loan Repayment': 9, 'Miscellaneous': 10, 'Rent / Mortgage': 11, 'Shopping': 12, 'Transport & Fuel': 13}}


In [205]:
# Create a copy to use for the correlation matrix (as it only accepts numerical data)
train_data_corr = train.copy()

In [206]:
# Replace the categories with the numerical equivalents from the dictionary
train_data_corr.replace(replace_map_train, inplace=True)

In [207]:
train_data_corr.head(3)

,MERCHANT_CATEGORIZED_AT,MERCHANT_NAME,MERCHANT_CATEGORIZED_AS,PURCHASE_VALUE,PURCHASED_AT,IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY,USER_AGE,USER_GENDER,USER_HOUSEHOLD,USER_INCOME,USER_ID,Transaction_ID
0,2022-05-04 10:25:50.588042+00,UONSDA CHURCH MOGERE MWAYO TITHE,11,1700,2022-05-04 13:56:00+00,False,25.803279,Male,3,10000,ID_ZX4DCF4K,ID_04mk78fa
1,2021-10-25 16:18:38.586837+00,PARK N GO,13,100,2021-10-24 14:12:00+00,False,25.000000,Female,4,90000,ID_U9WZMGJZ,ID_04xkfb07
2,2022-05-20 14:17:30.917297+00,SAFARICOM OFFERS TUNUKIWA,2,20,2022-05-27 12:46:00+00,False,25.803279,Male,3,10000,ID_ZX4DCF4K,ID_051urwuw


# One-hot encode

 One-hot encode MERCHANT_NAME

In [208]:

train["train"] = 1
test["train"] = 0

In [209]:
all_data = pd.concat([train, test])

In [210]:
all_data = pd.get_dummies(all_data, prefix_sep="_", columns=['MERCHANT_NAME'])

In [211]:
train = all_data[all_data["train"] == 1]
test = all_data[all_data["train"] == 0]

In [212]:
print("Train: ", train.shape)
print("Test: ", test.shape)

Train:  (373, 452)
Test:  (558, 452)


In [213]:
all_data.sample(n=3)

,MERCHANT_CATEGORIZED_AT,MERCHANT_CATEGORIZED_AS,PURCHASE_VALUE,PURCHASED_AT,IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY,USER_AGE,USER_GENDER,USER_HOUSEHOLD,USER_INCOME,USER_ID,Transaction_ID,train,MERCHANT_NAME_ABSA BANK KENYA PLC,MERCHANT_NAME_ACCESS AFYA,MERCHANT_NAME_AGGREY MWANGALE,MERCHANT_NAME_AGNES KASYOKI,MERCHANT_NAME_AGNES NGANGA,MERCHANT_NAME_ALBERT MBOGO,MERCHANT_NAME_ALCHEMIIST - LEGEND BAR,MERCHANT_NAME_ALEX SAKWA,MERCHANT_NAME_ALLAN WAMBURA,MERCHANT_NAME_ALLSTAR GAS SUPPLIES,MERCHANT_NAME_ALMA APARTMENTS MANAGEMENT CO LTD,MERCHANT_NAME_ALPHA CAR WASH,MERCHANT_NAME_AMBROSE MWALILI,MERCHANT_NAME_ANASTACIA CHEROBON,MERCHANT_NAME_ANKOLE GRILL LIMITED,MERCHANT_NAME_ANTONY MAKAU,MERCHANT_NAME_APS ABC PARKING,MERCHANT_NAME_ARDEN AXIDA ABUOCH,MERCHANT_NAME_ARNELLE OMONDI,MERCHANT_NAME_ARNICA PHARMACEUTICALS,MERCHANT_NAME_ART MARKET KILELESHWA,MERCHANT_NAME_ARTCAFFE TWO RIVERS,MERCHANT_NAME_ASHAKI GRILL AND BBQ,MERCHANT_NAME_ASL TD,MERCHANT_NAME_AUDRINE MAKAKA,MERCHANT_NAME_BAABAX EXPRESS LIMITED,MERCHANT_NAME_BARISTAPRO KENYA LTD,MERCHANT_NAME_BATA,MERCHANT_NAME_BAYES,MERCHANT_NAME_BERNARD NDUNG'U,MERCHANT_NAME_BETH GICHU,MERCHANT_NAME_BETHSEBA MAKOKHA,MERCHANT_NAME_BETHSHEBA NYAUNDI,MERCHANT_NAME_BIG FISH,MERCHANT_NAME_BILLY OMOLO,MERCHANT_NAME_BLIZZ WINES AND SPIRITS DISTRIBUTORS,MERCHANT_NAME_BOLT FASHIONS,MERCHANT_NAME_BRAVINE SUNGAH STOWER,MERCHANT_NAME_BRIAN MUNANGWE,MERCHANT_NAME_BRIAN OKUKU,MERCHANT_NAME_BRIOCHE RUBIS LANGATA,MERCHANT_NAME_BUY DIRECT LIMITED 2,MERCHANT_NAME_CAFETERIA AND TIN TIN RESTAURANT,MERCHANT_NAME_CAKES BY EUDIE LIMITED,MERCHANT_NAME_CAPTON ENTERPRISES NRBI WEST,MERCHANT_NAME_CAPTURE ADVENTURES,MERCHANT_NAME_CAROLINE KURIA,MERCHANT_NAME_CARREFOUR,MERCHANT_NAME_CARREFOUR GLR,MERCHANT_NAME_CARREFOUR GLR,MERCHANT_NAME_CARREFOUR SRT,MERCHANT_NAME_CASH NOW CASHNOW,MERCHANT_NAME_CBD-680,MERCHANT_NAME_CECILIA MWANGI,MERCHANT_NAME_CEDRIC OCHOLA,MERCHANT_NAME_CETRICK MUKALAVAI,MERCHANT_NAME_CHANDARANA YAYA 1,MERCHANT_NAME_CHAPA COPY COMPANY - KILELESHWA,MERCHANT_NAME_CHARLENNE AKETCH OBBO ONYANGO,MERCHANT_NAME_CHRISPIN MALOBA,MERCHANT_NAME_CHRISTINE MUTISYA,MERCHANT_NAME_CHRISTINE OOKO,MERCHANT_NAME_CHRISTOHER ONGAYA,MERCHANT_NAME_CITAM BUSINESS FORUM CBF ON,MERCHANT_NAME_CITAM KAREN TITHE ON,MERCHANT_NAME_CITAM KITENGELA OFFERING ON,MERCHANT_NAME_CLAPPERTON MUSYOKI,MERCHANT_NAME_CLENICE MBECHE,MERCHANT_NAME_CO-OPERATIVE BANK COLLECTION ACCOUNT,MERCHANT_NAME_COCO JAMBO,MERCHANT_NAME_COLLINS OUMA,MERCHANT_NAME_CRAFT - CHUPA CHAP,MERCHANT_NAME_CREDITAREA CAPITAL LIMITED,MERCHANT_NAME_DAVID MWANIKI NJOKA,MERCHANT_NAME_DECATHLON HUB TILL 5,MERCHANT_NAME_DEEJINA LIMITED,MERCHANT_NAME_DEEPAL PATEL,MERCHANT_NAME_DENIS OCHIENG OTIENO,MERCHANT_NAME_DENIS OTIENO,MERCHANT_NAME_DESMOND MATIMU,MERCHANT_NAME_DIRECT PAY,MERCHANT_NAME_DORCAS WANJOHI,MERCHANT_NAME_DORINE ADHIAMBO,MERCHANT_NAME_DR CECILIA,MERCHANT_NAME_DUKE NYAMARI,MERCHANT_NAME_DUNCAN NDUGO,MERCHANT_NAME_DUNTON OKOTH,MERCHANT_NAME_EDWIN OMBATI,MERCHANT_NAME_ELIAS MBURU,MERCHANT_NAME_ELIVIN KWAMBOKA MOYWAYWA,MERCHANT_NAME_ELIZABETH NYAMBURA,MERCHANT_NAME_ELIZABETH WANGARI,MERCHANT_NAME_ENGEN KENYA,MERCHANT_NAME_ENOCK MAKORI,MERCHANT_NAME_ENVY VIVA CORPORATE,MERCHANT_NAME_EQUITY PAYBILL,MERCHANT_NAME_EQUITY PAYBILL ACCOUNT,MERCHANT_NAME_ERIC CHESI,MERCHANT_NAME_ESTHER NDIRITU,MERCHANT_NAME_ESTHER WANJIKU KARANJA,MERCHANT_NAME_FAHARI RETIREMENT PLAN,MERCHANT_NAME_FAMILY BANK PESA PAP,MERCHANT_NAME_FARMER'S CHOICE STOCKISTS,MERCHANT_NAME_FARMERS BUTCHERY,MERCHANT_NAME_FATHALLA ABOUBAKR,MERCHANT_NAME_FATUMA OSORE,MERCHANT_NAME_FAYAZ BAKERS KILIFI TOWN SHOP,MERCHANT_NAME_FELIX KAINDI,MERCHANT_NAME_FISH BAY,MERCHANT_NAME_FLORENCE KANANA,MERCHANT_NAME_FOUR SQUARE FINE CUTS,MERCHANT_NAME_FRACIAH KAGAI,MERCHANT_NAME_FRANCIS KABOCHI,MERCHANT_NAME_FREDRICK NGATIA,MERCHANT_NAME_FREDRICK OMENYO,MERCHANT_NAME_FRIDAH OWINO,MERCHANT_NAME_FYTY CARWASH,MERCHANT_NAME_GALITOS BELLEVUE,MERCHANT_NAME_GARIDON ENTERPRISES VIA CO-OP,MERCHANT_NAME_GEORGE GACHERU,MERCHANT_NAME_GEO

## Drop unnecessary columns

In [214]:
train = train.drop(['MERCHANT_CATEGORIZED_AT','PURCHASED_AT','USER_ID', 'Transaction_ID', "train"], axis=1)
test = test.drop(['MERCHANT_CATEGORIZED_AT','PURCHASED_AT','USER_ID', "train", "MERCHANT_CATEGORIZED_AS"], axis=1)

# Convert Boolean to Binary

In [215]:
# Based on our data set, this function converts boolean to binary entries
def create_binary_cols(content):
  if content == False:
    content = 0
  elif content == True:
    content = 1
  elif content == 'N':
    content = 0
  elif content == 'Y':
    content = 1
  elif content == 'Male':
    content = 0
  elif content == 'Female':
    content = 1
  return content

In [216]:
# Gender column convert:
train['USER_GENDER'] = train['USER_GENDER'].apply(create_binary_cols)
test['USER_GENDER'] = test['USER_GENDER'].apply(create_binary_cols)

# Is_purchase_paid_via_mpesa_send_money column convert:
train['IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY'] = train['IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY'].apply(create_binary_cols)
test['IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY'] = test['IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY'].apply(create_binary_cols)

In [217]:
train.sample(n=3)

,MERCHANT_CATEGORIZED_AS,PURCHASE_VALUE,IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY,USER_AGE,USER_GENDER,USER_HOUSEHOLD,USER_INCOME,MERCHANT_NAME_ABSA BANK KENYA PLC,MERCHANT_NAME_ACCESS AFYA,MERCHANT_NAME_AGGREY MWANGALE,MERCHANT_NAME_AGNES KASYOKI,MERCHANT_NAME_AGNES NGANGA,MERCHANT_NAME_ALBERT MBOGO,MERCHANT_NAME_ALCHEMIIST - LEGEND BAR,MERCHANT_NAME_ALEX SAKWA,MERCHANT_NAME_ALLAN WAMBURA,MERCHANT_NAME_ALLSTAR GAS SUPPLIES,MERCHANT_NAME_ALMA APARTMENTS MANAGEMENT CO LTD,MERCHANT_NAME_ALPHA CAR WASH,MERCHANT_NAME_AMBROSE MWALILI,MERCHANT_NAME_ANASTACIA CHEROBON,MERCHANT_NAME_ANKOLE GRILL LIMITED,MERCHANT_NAME_ANTONY MAKAU,MERCHANT_NAME_APS ABC PARKING,MERCHANT_NAME_ARDEN AXIDA ABUOCH,MERCHANT_NAME_ARNELLE OMONDI,MERCHANT_NAME_ARNICA PHARMACEUTICALS,MERCHANT_NAME_ART MARKET KILELESHWA,MERCHANT_NAME_ARTCAFFE TWO RIVERS,MERCHANT_NAME_ASHAKI GRILL AND BBQ,MERCHANT_NAME_ASL TD,MERCHANT_NAME_AUDRINE MAKAKA,MERCHANT_NAME_BAABAX EXPRESS LIMITED,MERCHANT_NAME_BARISTAPRO KENYA LTD,MERCHANT_NAME_BATA,MERCHANT_NAME_BAYES,MERCHANT_NAME_BERNARD NDUNG'U,MERCHANT_NAME_BETH GICHU,MERCHANT_NAME_BETHSEBA MAKOKHA,MERCHANT_NAME_BETHSHEBA NYAUNDI,MERCHANT_NAME_BIG FISH,MERCHANT_NAME_BILLY OMOLO,MERCHANT_NAME_BLIZZ WINES AND SPIRITS DISTRIBUTORS,MERCHANT_NAME_BOLT FASHIONS,MERCHANT_NAME_BRAVINE SUNGAH STOWER,MERCHANT_NAME_BRIAN MUNANGWE,MERCHANT_NAME_BRIAN OKUKU,MERCHANT_NAME_BRIOCHE RUBIS LANGATA,MERCHANT_NAME_BUY DIRECT LIMITED 2,MERCHANT_NAME_CAFETERIA AND TIN TIN RESTAURANT,MERCHANT_NAME_CAKES BY EUDIE LIMITED,MERCHANT_NAME_CAPTON ENTERPRISES NRBI WEST,MERCHANT_NAME_CAPTURE ADVENTURES,MERCHANT_NAME_CAROLINE KURIA,MERCHANT_NAME_CARREFOUR,MERCHANT_NAME_CARREFOUR GLR,MERCHANT_NAME_CARREFOUR GLR,MERCHANT_NAME_CARREFOUR SRT,MERCHANT_NAME_CASH NOW CASHNOW,MERCHANT_NAME_CBD-680,MERCHANT_NAME_CECILIA MWANGI,MERCHANT_NAME_CEDRIC OCHOLA,MERCHANT_NAME_CETRICK MUKALAVAI,MERCHANT_NAME_CHANDARANA YAYA 1,MERCHANT_NAME_CHAPA COPY COMPANY - KILELESHWA,MERCHANT_NAME_CHARLENNE AKETCH OBBO ONYANGO,MERCHANT_NAME_CHRISPIN MALOBA,MERCHANT_NAME_CHRISTINE MUTISYA,MERCHANT_NAME_CHRISTINE OOKO,MERCHANT_NAME_CHRISTOHER ONGAYA,MERCHANT_NAME_CITAM BUSINESS FORUM CBF ON,MERCHANT_NAME_CITAM KAREN TITHE ON,MERCHANT_NAME_CITAM KITENGELA OFFERING ON,MERCHANT_NAME_CLAPPERTON MUSYOKI,MERCHANT_NAME_CLENICE MBECHE,MERCHANT_NAME_CO-OPERATIVE BANK COLLECTION ACCOUNT,MERCHANT_NAME_COCO JAMBO,MERCHANT_NAME_COLLINS OUMA,MERCHANT_NAME_CRAFT - CHUPA CHAP,MERCHANT_NAME_CREDITAREA CAPITAL LIMITED,MERCHANT_NAME_DAVID MWANIKI NJOKA,MERCHANT_NAME_DECATHLON HUB TILL 5,MERCHANT_NAME_DEEJINA LIMITED,MERCHANT_NAME_DEEPAL PATEL,MERCHANT_NAME_DENIS OCHIENG OTIENO,MERCHANT_NAME_DENIS OTIENO,MERCHANT_NAME_DESMOND MATIMU,MERCHANT_NAME_DIRECT PAY,MERCHANT_NAME_DORCAS WANJOHI,MERCHANT_NAME_DORINE ADHIAMBO,MERCHANT_NAME_DR CECILIA,MERCHANT_NAME_DUKE NYAMARI,MERCHANT_NAME_DUNCAN NDUGO,MERCHANT_NAME_DUNTON OKOTH,MERCHANT_NAME_EDWIN OMBATI,MERCHANT_NAME_ELIAS MBURU,MERCHANT_NAME_ELIVIN KWAMBOKA MOYWAYWA,MERCHANT_NAME_ELIZABETH NYAMBURA,MERCHANT_NAME_ELIZABETH WANGARI,MERCHANT_NAME_ENGEN KENYA,MERCHANT_NAME_ENOCK MAKORI,MERCHANT_NAME_ENVY VIVA CORPORATE,MERCHANT_NAME_EQUITY PAYBILL,MERCHANT_NAME_EQUITY PAYBILL ACCOUNT,MERCHANT_NAME_ERIC CHESI,MERCHANT_NAME_ESTHER NDIRITU,MERCHANT_NAME_ESTHER WANJIKU KARANJA,MERCHANT_NAME_FAHARI RETIREMENT PLAN,MERCHANT_NAME_FAMILY BANK PESA PAP,MERCHANT_NAME_FARMER'S CHOICE STOCKISTS,MERCHANT_NAME_FARMERS BUTCHERY,MERCHANT_NAME_FATHALLA ABOUBAKR,MERCHANT_NAME_FATUMA OSORE,MERCHANT_NAME_FAYAZ BAKERS KILIFI TOWN SHOP,MERCHANT_NAME_FELIX KAINDI,MERCHANT_NAME_FISH BAY,MERCHANT_NAME_FLORENCE KANANA,MERCHANT_NAME_FOUR SQUARE FINE CUTS,MERCHANT_NAME_FRACIAH KAGAI,MERCHANT_NAME_FRANCIS KABOCHI,MERCHANT_NAME_FREDRICK NGATIA,MERCHANT_NAME_FREDRICK OMENYO,MERCHANT_NAME_FRIDAH OWINO,MERCHANT_NAME_FYTY CARWASH,MERCHANT_NAME_GALITOS BELLEVUE,MERCHANT_NAME_GARIDON ENTERPRISES VIA CO-OP,MERCHANT_NAME_GEORGE GACHERU,MERCHANT_NAME_GEORGE KAMAU,MERCHANT_NAME_GHARIB HABIB,MERCHANT_NAME_GITHUNGURI SELF

# machine learning

In [218]:
caty.value_counts()

MERCHANT_CATEGORIZED_AS
Bills & Fees               78
Groceries                  50
Data & WiFi                43
Miscellaneous              43
Family & Friends           41
Going out                  41
Transport & Fuel           29
Shopping                   21
Emergency fund             12
Health                      6
Loan Repayment              5
Education                   2
Rent / Mortgage             2
dtype: int64

In [219]:
#adding the Education , Rent / Mortgage 
train2 = train[train["MERCHANT_CATEGORIZED_AS"].isin(["Education", "Rent / Mortgage"])]
train2 = pd.concat([train2]*5)
train2.head(10)

,MERCHANT_CATEGORIZED_AS,PURCHASE_VALUE,IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY,USER_AGE,USER_GENDER,USER_HOUSEHOLD,USER_INCOME,MERCHANT_NAME_ABSA BANK KENYA PLC,MERCHANT_NAME_ACCESS AFYA,MERCHANT_NAME_AGGREY MWANGALE,MERCHANT_NAME_AGNES KASYOKI,MERCHANT_NAME_AGNES NGANGA,MERCHANT_NAME_ALBERT MBOGO,MERCHANT_NAME_ALCHEMIIST - LEGEND BAR,MERCHANT_NAME_ALEX SAKWA,MERCHANT_NAME_ALLAN WAMBURA,MERCHANT_NAME_ALLSTAR GAS SUPPLIES,MERCHANT_NAME_ALMA APARTMENTS MANAGEMENT CO LTD,MERCHANT_NAME_ALPHA CAR WASH,MERCHANT_NAME_AMBROSE MWALILI,MERCHANT_NAME_ANASTACIA CHEROBON,MERCHANT_NAME_ANKOLE GRILL LIMITED,MERCHANT_NAME_ANTONY MAKAU,MERCHANT_NAME_APS ABC PARKING,MERCHANT_NAME_ARDEN AXIDA ABUOCH,MERCHANT_NAME_ARNELLE OMONDI,MERCHANT_NAME_ARNICA PHARMACEUTICALS,MERCHANT_NAME_ART MARKET KILELESHWA,MERCHANT_NAME_ARTCAFFE TWO RIVERS,MERCHANT_NAME_ASHAKI GRILL AND BBQ,MERCHANT_NAME_ASL TD,MERCHANT_NAME_AUDRINE MAKAKA,MERCHANT_NAME_BAABAX EXPRESS LIMITED,MERCHANT_NAME_BARISTAPRO KENYA LTD,MERCHANT_NAME_BATA,MERCHANT_NAME_BAYES,MERCHANT_NAME_BERNARD NDUNG'U,MERCHANT_NAME_BETH GICHU,MERCHANT_NAME_BETHSEBA MAKOKHA,MERCHANT_NAME_BETHSHEBA NYAUNDI,MERCHANT_NAME_BIG FISH,MERCHANT_NAME_BILLY OMOLO,MERCHANT_NAME_BLIZZ WINES AND SPIRITS DISTRIBUTORS,MERCHANT_NAME_BOLT FASHIONS,MERCHANT_NAME_BRAVINE SUNGAH STOWER,MERCHANT_NAME_BRIAN MUNANGWE,MERCHANT_NAME_BRIAN OKUKU,MERCHANT_NAME_BRIOCHE RUBIS LANGATA,MERCHANT_NAME_BUY DIRECT LIMITED 2,MERCHANT_NAME_CAFETERIA AND TIN TIN RESTAURANT,MERCHANT_NAME_CAKES BY EUDIE LIMITED,MERCHANT_NAME_CAPTON ENTERPRISES NRBI WEST,MERCHANT_NAME_CAPTURE ADVENTURES,MERCHANT_NAME_CAROLINE KURIA,MERCHANT_NAME_CARREFOUR,MERCHANT_NAME_CARREFOUR GLR,MERCHANT_NAME_CARREFOUR GLR,MERCHANT_NAME_CARREFOUR SRT,MERCHANT_NAME_CASH NOW CASHNOW,MERCHANT_NAME_CBD-680,MERCHANT_NAME_CECILIA MWANGI,MERCHANT_NAME_CEDRIC OCHOLA,MERCHANT_NAME_CETRICK MUKALAVAI,MERCHANT_NAME_CHANDARANA YAYA 1,MERCHANT_NAME_CHAPA COPY COMPANY - KILELESHWA,MERCHANT_NAME_CHARLENNE AKETCH OBBO ONYANGO,MERCHANT_NAME_CHRISPIN MALOBA,MERCHANT_NAME_CHRISTINE MUTISYA,MERCHANT_NAME_CHRISTINE OOKO,MERCHANT_NAME_CHRISTOHER ONGAYA,MERCHANT_NAME_CITAM BUSINESS FORUM CBF ON,MERCHANT_NAME_CITAM KAREN TITHE ON,MERCHANT_NAME_CITAM KITENGELA OFFERING ON,MERCHANT_NAME_CLAPPERTON MUSYOKI,MERCHANT_NAME_CLENICE MBECHE,MERCHANT_NAME_CO-OPERATIVE BANK COLLECTION ACCOUNT,MERCHANT_NAME_COCO JAMBO,MERCHANT_NAME_COLLINS OUMA,MERCHANT_NAME_CRAFT - CHUPA CHAP,MERCHANT_NAME_CREDITAREA CAPITAL LIMITED,MERCHANT_NAME_DAVID MWANIKI NJOKA,MERCHANT_NAME_DECATHLON HUB TILL 5,MERCHANT_NAME_DEEJINA LIMITED,MERCHANT_NAME_DEEPAL PATEL,MERCHANT_NAME_DENIS OCHIENG OTIENO,MERCHANT_NAME_DENIS OTIENO,MERCHANT_NAME_DESMOND MATIMU,MERCHANT_NAME_DIRECT PAY,MERCHANT_NAME_DORCAS WANJOHI,MERCHANT_NAME_DORINE ADHIAMBO,MERCHANT_NAME_DR CECILIA,MERCHANT_NAME_DUKE NYAMARI,MERCHANT_NAME_DUNCAN NDUGO,MERCHANT_NAME_DUNTON OKOTH,MERCHANT_NAME_EDWIN OMBATI,MERCHANT_NAME_ELIAS MBURU,MERCHANT_NAME_ELIVIN KWAMBOKA MOYWAYWA,MERCHANT_NAME_ELIZABETH NYAMBURA,MERCHANT_NAME_ELIZABETH WANGARI,MERCHANT_NAME_ENGEN KENYA,MERCHANT_NAME_ENOCK MAKORI,MERCHANT_NAME_ENVY VIVA CORPORATE,MERCHANT_NAME_EQUITY PAYBILL,MERCHANT_NAME_EQUITY PAYBILL ACCOUNT,MERCHANT_NAME_ERIC CHESI,MERCHANT_NAME_ESTHER NDIRITU,MERCHANT_NAME_ESTHER WANJIKU KARANJA,MERCHANT_NAME_FAHARI RETIREMENT PLAN,MERCHANT_NAME_FAMILY BANK PESA PAP,MERCHANT_NAME_FARMER'S CHOICE STOCKISTS,MERCHANT_NAME_FARMERS BUTCHERY,MERCHANT_NAME_FATHALLA ABOUBAKR,MERCHANT_NAME_FATUMA OSORE,MERCHANT_NAME_FAYAZ BAKERS KILIFI TOWN SHOP,MERCHANT_NAME_FELIX KAINDI,MERCHANT_NAME_FISH BAY,MERCHANT_NAME_FLORENCE KANANA,MERCHANT_NAME_FOUR SQUARE FINE CUTS,MERCHANT_NAME_FRACIAH KAGAI,MERCHANT_NAME_FRANCIS KABOCHI,MERCHANT_NAME_FREDRICK NGATIA,MERCHANT_NAME_FREDRICK OMENYO,MERCHANT_NAME_FRIDAH OWINO,MERCHANT_NAME_FYTY CARWASH,MERCHANT_NAME_GALITOS BELLEVUE,MERCHANT_NAME_GARIDON ENTERPRISES VIA CO-OP,MERCHANT_NAME_GEORGE GACHERU,MERCHANT_NAME_GEORGE KAMAU,MERCHANT_NAME_GHARIB HABIB,MERCHANT_NAME_GITHUNGURI SELF

In [220]:
#merge train 2 and train
train = pd.concat([train, train2])


In [221]:
# Separate the features from the target in the training data
X = train.drop(["MERCHANT_CATEGORIZED_AS"], axis=1)
y = train["MERCHANT_CATEGORIZED_AS"]

In [222]:
y.nunique()

13

In [223]:
X.shape

(393, 446)

In [224]:
# Split the training data

X_train, X_val, y_train, y_val = train_test_split(train.drop(["MERCHANT_CATEGORIZED_AS"], axis=1), train["MERCHANT_CATEGORIZED_AS"], 
                                                  stratify=train[["MERCHANT_CATEGORIZED_AS"]],  test_size=0.1, random_state=101)

In [225]:
len(y_train.unique()), len(y_val.unique())

(13, 13)

# logloss

In [247]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

# 1. decision tree

In [251]:
# Creating a Decision tree classifier
model_dtree = DecisionTreeClassifier(max_depth=5, random_state=101,
                                     max_features = None, min_samples_leaf=8)

# Training the model on the training dataset
model_dtree.fit(X_train, y_train)

# Predict
y_pred = model_dtree.predict(X_val)

# Calculate accuracy
print("ACCURACY OF THE MODEL: ", accuracy_score(y_val, y_pred))

# Classification report
print(classification_report(y_val, y_pred))

ACCURACY OF THE MODEL:  0.325
                  precision    recall  f1-score   support

    Bills & Fees       0.62      0.62      0.62         8
     Data & WiFi       0.50      0.25      0.33         4
       Education       0.00      0.00      0.00         1
  Emergency fund       0.00      0.00      0.00         1
Family & Friends       0.22      0.50      0.31         4
       Going out       0.20      0.50      0.29         4
       Groceries       0.20      0.40      0.27         5
          Health       0.00      0.00      0.00         1
  Loan Repayment       0.00      0.00      0.00         1
   Miscellaneous       1.00      0.20      0.33         5
 Rent / Mortgage       0.00      0.00      0.00         1
        Shopping       0.00      0.00      0.00         2
Transport & Fuel       0.00      0.00      0.00         3

        accuracy                           0.33        40
       macro avg       0.21      0.19      0.17        40
    weighted avg       0.37      0.33   

# 2. random forest with 100 estimators

In [254]:
from sklearn.ensemble import RandomForestClassifier

# Creating a RF classifier
clf = RandomForestClassifier() 
 
# Training the model on the training dataset
clf.fit(X_train, y_train)
 
# Performing predictions on the test dataset
y_pred = clf.predict(X_val)

# Calculate accuracy
print("ACCURACY OF THE MODEL: ", accuracy_score(y_val, y_pred))

# Classification report
print(classification_report(y_val, y_pred))


ACCURACY OF THE MODEL:  0.625
                  precision    recall  f1-score   support

    Bills & Fees       0.88      0.88      0.88         8
     Data & WiFi       0.67      1.00      0.80         4
       Education       0.50      1.00      0.67         1
  Emergency fund       0.00      0.00      0.00         1
Family & Friends       0.67      0.50      0.57         4
       Going out       0.33      0.50      0.40         4
       Groceries       0.60      0.60      0.60         5
          Health       0.00      0.00      0.00         1
  Loan Repayment       0.00      0.00      0.00         1
   Miscellaneous       0.75      0.60      0.67         5
 Rent / Mortgage       1.00      1.00      1.00         1
        Shopping       0.50      0.50      0.50         2
Transport & Fuel       0.33      0.33      0.33         3

        accuracy                           0.62        40
       macro avg       0.48      0.53      0.49        40
    weighted avg       0.60      0.62   

In [255]:
grid_search = {'criterion': ['entropy', 'gini'],
               'max_depth': [2],
               'max_features': ['auto', 'sqrt'],
               'min_samples_leaf': [4, 6, 8],
               'min_samples_split': [5, 7,10],
               'n_estimators': [20]}

model = GridSearchCV(estimator = clf, param_grid = grid_search, 
                               cv = 4, verbose= 5, n_jobs = -1)
model.fit(X_train, y_train)

# Performing predictions on the test dataset
y_pred = model.predict(X_val)

# Calculate accuracy
print("ACCURACY OF THE MODEL: ", accuracy_score(y_val, y_pred))

# Classification report
print(classification_report(y_val, y_pred))

Fitting 4 folds for each of 36 candidates, totalling 144 fits
ACCURACY OF THE MODEL:  0.25
                  precision    recall  f1-score   support

    Bills & Fees       0.22      1.00      0.36         8
     Data & WiFi       0.50      0.25      0.33         4
       Education       0.00      0.00      0.00         1
  Emergency fund       0.00      0.00      0.00         1
Family & Friends       0.00      0.00      0.00         4
       Going out       0.00      0.00      0.00         4
       Groceries       0.00      0.00      0.00         5
          Health       0.00      0.00      0.00         1
  Loan Repayment       0.00      0.00      0.00         1
   Miscellaneous       1.00      0.20      0.33         5
 Rent / Mortgage       0.00      0.00      0.00         1
        Shopping       0.00      0.00      0.00         2
Transport & Fuel       0.00      0.00      0.00         3

        accuracy                           0.25        40
       macro avg       0.13      0.11

In [256]:
!pip install hyperopt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [259]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

space = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.quniform('max_depth', 10, 12, 10),
        'max_features': hp.choice('max_features', ['auto', 'sqrt','log2', None]),
        'min_samples_leaf': hp.uniform ('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.choice('n_estimators', [10, 50])
    }

def objective(space):
    model = RandomForestClassifier(criterion = space['criterion'], 
                                   max_depth = space['max_depth'],
                                 max_features = space['max_features'],
                                 min_samples_leaf = space['min_samples_leaf'],
                                 min_samples_split = space['min_samples_split'],
                                 n_estimators = space['n_estimators'], 
                                 )
    
    accuracy = cross_val_score(model, X_train, y_train, cv = 4).mean()

    # We aim to maximize accuracy, therefore we return it as a negative value
    return {'loss': -accuracy, 'status': STATUS_OK }
    
trials = Trials()

best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 20,
            trials= trials)
best

100%|██████████| 20/20 [00:05<00:00,  3.72it/s, best loss: -0.21811159346271708]


{'criterion': 0,
 'max_depth': 10.0,
 'max_features': 3,
 'min_samples_leaf': 0.2510775947184087,
 'min_samples_split': 0.1443840643797707,
 'n_estimators': 0}

In [273]:
crit = {0: 'entropy', 1: 'gini'}
feat = {0: 'auto', 1: 'sqrt', 2: 'log2', 3: None}
est = {0: 10, 1: 50, 2: 75, 3: 100, 4: 125}

clf = RandomForestClassifier(criterion = crit[best['criterion']], 
                                       max_depth = best['max_depth'], 
                                       max_features = feat[best['max_features']], 
                                       min_samples_leaf = best['min_samples_leaf'], 
                                       min_samples_split = best['min_samples_split'], 
                                       n_estimators = est[best['n_estimators']]
                                      )
# Creating a RF classifier
clf = RandomForestClassifier() 
 
# Training the model on the training dataset
clf.fit(X_train, y_train)
 
# Performing predictions on the test dataset
y_pred = clf.predict(X_val)

# Calculate accuracy
print("ACCURACY OF THE MODEL: ", accuracy_score(y_val, y_pred))

# Classification report
print(classification_report(y_val, y_pred))
                                    

ACCURACY OF THE MODEL:  0.6
                  precision    recall  f1-score   support

    Bills & Fees       0.80      1.00      0.89         8
     Data & WiFi       0.67      1.00      0.80         4
       Education       0.50      1.00      0.67         1
  Emergency fund       0.00      0.00      0.00         1
Family & Friends       0.33      0.25      0.29         4
       Going out       0.33      0.50      0.40         4
       Groceries       0.60      0.60      0.60         5
          Health       0.00      0.00      0.00         1
  Loan Repayment       0.00      0.00      0.00         1
   Miscellaneous       0.75      0.60      0.67         5
 Rent / Mortgage       1.00      1.00      1.00         1
        Shopping       0.00      0.00      0.00         2
Transport & Fuel       0.50      0.33      0.40         3

        accuracy                           0.60        40
       macro avg       0.42      0.48      0.44        40
    weighted avg       0.54      0.60     

# 3. Xgboost

In [290]:
from xgboost import XGBClassifier

# Creating a RF classifier
'''clf =  XGBClassifier(learning_rate =0.1,
                     n_estimators=1000,
                      max_depth=5,
                      min_child_weight=1,
                      gamma=0,
                      subsample=0.8,
                      colsample_bytree=0.8,
                      objective= 'binary:logistic',
                      nthread=4,
                      scale_pos_weight=1,
                     seed=27)''' 
clf =  XGBClassifier(max_depth= 3, min_child_weight = 1)
 
# Training the model on the training dataset
clf.fit(X_train, y_train)
 
# Performing predictions on the test dataset
y_pred = clf.predict(X_val)

# Calculate accuracy
print("ACCURACY OF THE MODEL: ", accuracy_score(y_val, y_pred))

# Classification report
print(classification_report(y_val, y_pred))

ACCURACY OF THE MODEL:  0.5
                  precision    recall  f1-score   support

    Bills & Fees       0.75      0.75      0.75         8
     Data & WiFi       0.57      1.00      0.73         4
       Education       1.00      1.00      1.00         1
  Emergency fund       0.00      0.00      0.00         1
Family & Friends       0.33      0.25      0.29         4
       Going out       0.25      0.25      0.25         4
       Groceries       0.60      0.60      0.60         5
          Health       0.00      0.00      0.00         1
  Loan Repayment       0.00      0.00      0.00         1
   Miscellaneous       0.67      0.40      0.50         5
 Rent / Mortgage       1.00      1.00      1.00         1
        Shopping       0.00      0.00      0.00         2
Transport & Fuel       0.25      0.33      0.29         3

        accuracy                           0.50        40
       macro avg       0.42      0.43      0.42        40
    weighted avg       0.49      0.50     

In [287]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4, cv=5)
gsearch1.fit(X_train, y_train)


AttributeError: ignored

In [289]:
gsearch1.best_params_, gsearch1.best_score_

({'max_depth': 3, 'min_child_weight': 1}, nan)

# 4. SVM

# Artificial Neural Networks (ANNs) Tuning


In [269]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf

In [270]:
def DL_Model(activation= 'linear', neurons= 5, optimizer='Adam'):
    model = Sequential()
    model.add(Dense(neurons, input_dim= 4, activation= activation))
    model.add(Dense(neurons, activation= activation))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer= optimizer, metrics=['accuracy'])
    return model

In [271]:
# Defining grid parameters
activation = ['softmax', 'relu']
neurons = [5, 10]
optimizer = ['Adam', 'Adamax']
param_grid = dict(activation = activation, neurons = neurons, optimizer = optimizer)

clf = KerasClassifier(build_fn= DL_Model, epochs= 5, batch_size=1024, verbose= 2)

In [272]:
model = GridSearchCV(estimator= clf, param_grid=param_grid, n_jobs=-1)
model.fit(X_train, y_train)

TypeError: ignored

In [ ]:
print("Max Accuracy Registred: {} using {}".format(round(model.best_score_,3), 
                                                   model.best_params_))

# hyperparameter optimizations

Grid Search

first create a scoring function

In [ ]:
# We will use logistic regression here..
lr_model = LogisticRegression()
# Random Forest
rf_model = RandomForestClassifier(n_jobs=-1, random_state=42)

# Create the pipeline 
clf = pipeline.Pipeline([('svd', svd),
                         ('scl', scl),
                         ('lr', lr_model)])

In [253]:
param_grid = {'lr__C': [0.1, 1.0, 10], 
              'lr__penalty': ['l1', 'l2']}

# Making the submission

In [227]:
labels = ['Bills & Fees',
 'Data & WiFi',
 'Education',
 'Emergency fund',
 'Family & Friends',
 'Going out',
 'Groceries',
 'Health',
 'Loan Repayment',
 'Miscellaneous',
 'Rent / Mortgage',
 'Shopping',
 'Transport & Fuel']

In [228]:
test.head(2)

,PURCHASE_VALUE,IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY,USER_AGE,USER_GENDER,USER_HOUSEHOLD,USER_INCOME,Transaction_ID,MERCHANT_NAME_ABSA BANK KENYA PLC,MERCHANT_NAME_ACCESS AFYA,MERCHANT_NAME_AGGREY MWANGALE,MERCHANT_NAME_AGNES KASYOKI,MERCHANT_NAME_AGNES NGANGA,MERCHANT_NAME_ALBERT MBOGO,MERCHANT_NAME_ALCHEMIIST - LEGEND BAR,MERCHANT_NAME_ALEX SAKWA,MERCHANT_NAME_ALLAN WAMBURA,MERCHANT_NAME_ALLSTAR GAS SUPPLIES,MERCHANT_NAME_ALMA APARTMENTS MANAGEMENT CO LTD,MERCHANT_NAME_ALPHA CAR WASH,MERCHANT_NAME_AMBROSE MWALILI,MERCHANT_NAME_ANASTACIA CHEROBON,MERCHANT_NAME_ANKOLE GRILL LIMITED,MERCHANT_NAME_ANTONY MAKAU,MERCHANT_NAME_APS ABC PARKING,MERCHANT_NAME_ARDEN AXIDA ABUOCH,MERCHANT_NAME_ARNELLE OMONDI,MERCHANT_NAME_ARNICA PHARMACEUTICALS,MERCHANT_NAME_ART MARKET KILELESHWA,MERCHANT_NAME_ARTCAFFE TWO RIVERS,MERCHANT_NAME_ASHAKI GRILL AND BBQ,MERCHANT_NAME_ASL TD,MERCHANT_NAME_AUDRINE MAKAKA,MERCHANT_NAME_BAABAX EXPRESS LIMITED,MERCHANT_NAME_BARISTAPRO KENYA LTD,MERCHANT_NAME_BATA,MERCHANT_NAME_BAYES,MERCHANT_NAME_BERNARD NDUNG'U,MERCHANT_NAME_BETH GICHU,MERCHANT_NAME_BETHSEBA MAKOKHA,MERCHANT_NAME_BETHSHEBA NYAUNDI,MERCHANT_NAME_BIG FISH,MERCHANT_NAME_BILLY OMOLO,MERCHANT_NAME_BLIZZ WINES AND SPIRITS DISTRIBUTORS,MERCHANT_NAME_BOLT FASHIONS,MERCHANT_NAME_BRAVINE SUNGAH STOWER,MERCHANT_NAME_BRIAN MUNANGWE,MERCHANT_NAME_BRIAN OKUKU,MERCHANT_NAME_BRIOCHE RUBIS LANGATA,MERCHANT_NAME_BUY DIRECT LIMITED 2,MERCHANT_NAME_CAFETERIA AND TIN TIN RESTAURANT,MERCHANT_NAME_CAKES BY EUDIE LIMITED,MERCHANT_NAME_CAPTON ENTERPRISES NRBI WEST,MERCHANT_NAME_CAPTURE ADVENTURES,MERCHANT_NAME_CAROLINE KURIA,MERCHANT_NAME_CARREFOUR,MERCHANT_NAME_CARREFOUR GLR,MERCHANT_NAME_CARREFOUR GLR,MERCHANT_NAME_CARREFOUR SRT,MERCHANT_NAME_CASH NOW CASHNOW,MERCHANT_NAME_CBD-680,MERCHANT_NAME_CECILIA MWANGI,MERCHANT_NAME_CEDRIC OCHOLA,MERCHANT_NAME_CETRICK MUKALAVAI,MERCHANT_NAME_CHANDARANA YAYA 1,MERCHANT_NAME_CHAPA COPY COMPANY - KILELESHWA,MERCHANT_NAME_CHARLENNE AKETCH OBBO ONYANGO,MERCHANT_NAME_CHRISPIN MALOBA,MERCHANT_NAME_CHRISTINE MUTISYA,MERCHANT_NAME_CHRISTINE OOKO,MERCHANT_NAME_CHRISTOHER ONGAYA,MERCHANT_NAME_CITAM BUSINESS FORUM CBF ON,MERCHANT_NAME_CITAM KAREN TITHE ON,MERCHANT_NAME_CITAM KITENGELA OFFERING ON,MERCHANT_NAME_CLAPPERTON MUSYOKI,MERCHANT_NAME_CLENICE MBECHE,MERCHANT_NAME_CO-OPERATIVE BANK COLLECTION ACCOUNT,MERCHANT_NAME_COCO JAMBO,MERCHANT_NAME_COLLINS OUMA,MERCHANT_NAME_CRAFT - CHUPA CHAP,MERCHANT_NAME_CREDITAREA CAPITAL LIMITED,MERCHANT_NAME_DAVID MWANIKI NJOKA,MERCHANT_NAME_DECATHLON HUB TILL 5,MERCHANT_NAME_DEEJINA LIMITED,MERCHANT_NAME_DEEPAL PATEL,MERCHANT_NAME_DENIS OCHIENG OTIENO,MERCHANT_NAME_DENIS OTIENO,MERCHANT_NAME_DESMOND MATIMU,MERCHANT_NAME_DIRECT PAY,MERCHANT_NAME_DORCAS WANJOHI,MERCHANT_NAME_DORINE ADHIAMBO,MERCHANT_NAME_DR CECILIA,MERCHANT_NAME_DUKE NYAMARI,MERCHANT_NAME_DUNCAN NDUGO,MERCHANT_NAME_DUNTON OKOTH,MERCHANT_NAME_EDWIN OMBATI,MERCHANT_NAME_ELIAS MBURU,MERCHANT_NAME_ELIVIN KWAMBOKA MOYWAYWA,MERCHANT_NAME_ELIZABETH NYAMBURA,MERCHANT_NAME_ELIZABETH WANGARI,MERCHANT_NAME_ENGEN KENYA,MERCHANT_NAME_ENOCK MAKORI,MERCHANT_NAME_ENVY VIVA CORPORATE,MERCHANT_NAME_EQUITY PAYBILL,MERCHANT_NAME_EQUITY PAYBILL ACCOUNT,MERCHANT_NAME_ERIC CHESI,MERCHANT_NAME_ESTHER NDIRITU,MERCHANT_NAME_ESTHER WANJIKU KARANJA,MERCHANT_NAME_FAHARI RETIREMENT PLAN,MERCHANT_NAME_FAMILY BANK PESA PAP,MERCHANT_NAME_FARMER'S CHOICE STOCKISTS,MERCHANT_NAME_FARMERS BUTCHERY,MERCHANT_NAME_FATHALLA ABOUBAKR,MERCHANT_NAME_FATUMA OSORE,MERCHANT_NAME_FAYAZ BAKERS KILIFI TOWN SHOP,MERCHANT_NAME_FELIX KAINDI,MERCHANT_NAME_FISH BAY,MERCHANT_NAME_FLORENCE KANANA,MERCHANT_NAME_FOUR SQUARE FINE CUTS,MERCHANT_NAME_FRACIAH KAGAI,MERCHANT_NAME_FRANCIS KABOCHI,MERCHANT_NAME_FREDRICK NGATIA,MERCHANT_NAME_FREDRICK OMENYO,MERCHANT_NAME_FRIDAH OWINO,MERCHANT_NAME_FYTY CARWASH,MERCHANT_NAME_GALITOS BELLEVUE,MERCHANT_NAME_GARIDON ENTERPRISES VIA CO-OP,MERCHANT_NAME_GEORGE GACHERU,MERCHANT_NAME_GEORGE KAMAU,MERCHANT_NAME_GHARIB HABIB,MERCHANT_NAME_GITHUNGURI SELFRIDGES SU

In [229]:
test1 = test.copy()

In [291]:
preds = clf.predict_proba(test.drop("Transaction_ID" , axis=1))
preds_df = pd.DataFrame(preds, columns=labels)
preds_df = pd.concat(objs=[test["Transaction_ID"], preds_df], axis="columns")

In [292]:
preds_df.sample(5)


,Transaction_ID,Bills & Fees,Data & WiFi,Education,Emergency fund,Family & Friends,Going out,Groceries,Health,Loan Repayment,Miscellaneous,Rent / Mortgage,Shopping,Transport & Fuel
337,ID_mdtw17tp,0.120871,0.044833,0.006358,0.003893,0.603579,0.022050,0.026075,0.003173,0.070074,0.058643,0.005170,0.028371,0.006911
319,ID_l5qd05ga,0.215258,0.043929,0.003324,0.017354,0.309974,0.059755,0.056153,0.006855,0.004952,0.109573,0.006926,0.113254,0.052692
336,ID_m7btsblj,0.087175,0.024920,0.002053,0.030785,0.012933,0.206049,0.297470,0.217358,0.001870,0.077625,0.001900,0.020204,0.019659
302,ID_k1uaj1ym,0.082688,0.019157,0.001917,0.001625,0.011768,0.360391,0.041659,0.048275,0.002452,0.021887,0.002927,0.321301,0.083954
94,ID_6d9wplkp,0.137812,0.022390,0.003519,0.001859,0.067448,0.119507,0.346271,0.007869,0.007222,0.055803,0.002646,0.075807,0.151846


In [293]:
preds_df.to_csv("submission6.csv", index=False)
